In [ ]:
# 5 lab
import re
import numpy as np

In [ ]:
# 5 lab
ids = [2049, 3587, 3588, 5, 3078, 2570, 3600, 2579, 4, 533, 3607, 1560, 3610, 2076, 30, 2591, 2457, 2084, 776, 46, 1586, 3123, 3125, 3639, 2621, 2622, 70, 2634, 587, 588, 2126, 1616, 3668, 1622, 88, 2649, 869, 3175, 1632, 1121, 1126, 1639, 189, 1644, 1647, 2238, 1651, 630, 3708, 3198, 3713, 1154, 3203, 2668, 1669, 2183, 1674, 3728, 2194, 147, 665, 1690, 624, 2206, 672, 1188, 678, 1626, 1704, 2731, 173, 1054, 180, 2230, 695, 3256, 1211, 701, 2411, 1728, 1217, 3780, 2758, 3783, 2253, 2763, 1741, 2255, 1150, 1747, 213, 3292, 3296, 636, 2280, 3265, 1406, 241, 3317, 758, 3833, 2816, 1281, 3332, 775, 1800, 1289, 3850, 2828, 783, 2323, 1305, 3420, 3376, 1311, 800, 1829, 2855, 808, 2861, 2864, 2353, 1331, 3892, 3551, 3722, 3387, 3808, 2878, 1344, 833, 2884, 2374, 3913, 336, 1361, 2389, 1367, 656, 1885, 2398, 1892, 357, 2406, 2450, 1387, 3438, 1911, 382, 1407, 1345, 2438, 1419, 1932, 909, 399, 913, 402, 917, 406, 409, 3824, 414, 1439, 3489, 2467, 421, 2984, 1961, 427, 3959, 1967, 3067, 2463, 3001, 955, 958, 715, 972, 978, 562, 3544, 3547, 1500, 3038, 479, 2215, 2535, 3562, 3564, 1006, 2544, 3570, 1013, 887, 3066, 2043, 3580, 510, 2134]

In [ ]:
def strip_html(input_str: str) -> str:
    clean = re.compile('<.*?>')
    return re.sub(clean, '', input_str)

In [ ]:
# загрузим все тексты в память и очистим от html тегов
base_texts = [] # 1: текст, 2: текст
for i in range(1, 21):
    with open('lab05data/base_{0}.txt'.format(i), 'r') as file:
        base_texts.append(strip_html(file.read()))

base_texts[:5]

In [ ]:
# load test texts
test_texts = {}
for i in ids:
    with open('lab05data/test_{0}.txt'.format(i), 'r') as file:
        txt = strip_html(file.read())
        test_texts[i] = txt

# top 5 records for checking
# {k: test_texts[k] for k in list(test_texts)[:5]}
[test_texts[k] for k in list(test_texts)][:5]

In [ ]:
# токенизируем слова
GROUPING_SPACE_REGEX = re.compile(r'([^\w]|[+])', re.UNICODE)
def simple_word_tokenize(text, _split=GROUPING_SPACE_REGEX.split):
    return [t for t in _split(text.lower()) if t and not t.isspace()]

In [ ]:
# import nltk
# import ssl
#
# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context
#
# nltk.download('stopwords')


In [ ]:
import pymorphy2
m = pymorphy2.MorphAnalyzer()
def token_r(text):
    var_words = simple_word_tokenize(text)
    return [m.parse(x)[0].normal_form for x in var_words if len(x) > 3]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=token_r)
matrix = cv.fit_transform(base_texts).toarray()

In [ ]:
# выделим стоп-слова и добавим 5 самых встречающихся
from nltk.corpus import stopwords
stops = set(stopwords.words('russian'))
words = [y[0] for y in sorted(cv.vocabulary_.items(), key=lambda x: x[1])]
c_words = zip(words, sum(matrix))
c_words = sorted(c_words, key=lambda x: x[1], reverse=True)
stop_w = set([x[0] for x in c_words[:5]])
stop_w = list(stop_w | stops)
str(stop_w)

In [ ]:
# token_r(base_texts[0])

In [ ]:
# переопределим каунт-векторайзер с учетом стоп-слов
cv = CountVectorizer(tokenizer=token_r, stop_words=stop_w)

In [ ]:
%%time
matrix = cv.fit_transform(base_texts).toarray()

In [ ]:
matrix.shape

In [ ]:
matrix_test = cv.transform([test_texts[k] for k in list(test_texts)])

In [ ]:
matrix_test[0]

In [ ]:
# просто тест косинусной близости на одном и том же тексте
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(matrix[0].reshape(1,-1), matrix[0].reshape(1,-1))


In [ ]:
# %matplotlib inline
# from matplotlib import pyplot as plt
# plt.figure(figsize=(17,10))
# plt.plot(cos_m[0][1:])

In [ ]:
# посчитаем данные
# берем по очереди каждый тестовый текст и считаем по нему косинусную меру для каждого из 20 базовых
test_scores = {} # test_id, [score_array]
for id_index in range(len(ids)):
    test_scores[ids[id_index]] = []
    for base_index in range(len(base_texts)):
        # посчитаем скоры
        score = cosine_similarity(matrix[base_index].reshape(1,-1), matrix_test[id_index])[0][0]
        test_scores[ids[id_index]].append(score)

{k: test_scores[k] for k in list(test_scores)[:1]}

In [ ]:
# посчитаем среднее со всех скоров
all_scores = []
for test_score_id in test_scores:
    for score in test_scores[test_score_id]:
        all_scores.append(score)
avg_value = np.mean(all_scores)
avg_value

In [ ]:
# посчитаем среднее по кажому айтему
test_scores_avg = {}
for test_score_id in test_scores:
    arr = test_scores[test_score_id]
    test_scores_avg[test_score_id] = np.mean(arr)

{k: test_scores_avg[k] for k in list(test_scores_avg)[:5]}

In [ ]:
# разделим выборку относительно этого среднего
good_ids = [k for k in list(test_scores_avg) if test_scores_avg[k] > avg_value]
bad_ids = [k for k in list(test_scores_avg) if test_scores_avg[k] <= avg_value]
(str(good_ids), str(bad_ids))
# руками копировал в json